In [2]:
import numpy as np
import pandas as pd
from utils import misc
from utils.misc import get_attr
import matplotlib.pyplot as plt

import time
pd.set_option('mode.chained_assignment', None)
from framework import tester, strategy
from environments.gym_env import TradingEnv



ModuleNotFoundError: No module named 'misc'

In [2]:
spy_df = pd.read_csv('stock_price_data/SPY.csv')
spy_df = misc.clean_df(spy_df)
spy_df

,date,open,high,low,close,adjclose,volume,ticker
0,1993-01-29,43.968750,43.968750,43.750000,43.937500,25.218206,1003200,SPY
1,1993-02-01,43.968750,44.250000,43.968750,44.250000,25.397572,480500,SPY
2,1993-02-02,44.218750,44.375000,44.125000,44.343750,25.451397,201300,SPY
3,1993-02-03,44.406250,44.843750,44.375000,44.812500,25.720430,529400,SPY
4,1993-02-04,44.968750,45.093750,44.468750,45.000000,25.828049,531500,SPY
...,...,...,...,...,...,...,...,...
7581,2023-03-09,399.739990,401.480011,390.529999,391.559998,391.559998,111945300,SPY
7582,2023-03-10,390.989990,393.160004,384.320007,385.910004,385.910004,189105300,SPY
7583,2023-03-13,381.809998,390.390015,380.649994,385.359985,385.359985,157790000,SPY
7584,2023-03-14,390.500000,393.450012,387.049988,391.730011,391.730011,149752400,SPY


In [3]:


def ma_signal(df, **kwargs):
    
    short_period = get_attr(kwargs, 'short_period', 10)
    long_period = get_attr(kwargs, 'long_period', 20)

    df['signal'] = df['adjclose'].rolling(short_period).mean() - df['adjclose'].rolling(long_period).mean()
    df['signal+1'] = df['signal'].shift(1)
    df = df.dropna()

    return df



df = spy_df.copy()
df['signal'] = df['adjclose'].rolling(10).mean() - df['adjclose'].rolling(20).mean()
df['signal+1'] = df['signal'].shift(1)
df['returns'] = np.insert(df['close'].to_numpy()[1:] / df['close'].to_numpy()[:-1], 1, 0) - 1
df['volatility'] = df['returns'].rolling(30).std()

df



# print(env.records['portfolio_volatility'])

,date,open,high,low,close,adjclose,volume,ticker,signal,signal+1,returns,volatility
0,1993-01-29,43.968750,43.968750,43.750000,43.937500,25.218206,1003200,SPY,NaN,NaN,0.007112,NaN
1,1993-02-01,43.968750,44.250000,43.968750,44.250000,25.397572,480500,SPY,NaN,NaN,-1.000000,NaN
2,1993-02-02,44.218750,44.375000,44.125000,44.343750,25.451397,201300,SPY,NaN,NaN,0.002119,NaN
3,1993-02-03,44.406250,44.843750,44.375000,44.812500,25.720430,529400,SPY,NaN,NaN,0.010571,NaN
4,1993-02-04,44.968750,45.093750,44.468750,45.000000,25.828049,531500,SPY,NaN,NaN,0.004184,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7581,2023-03-09,399.739990,401.480011,390.529999,391.559998,391.559998,111945300,SPY,-4.404001,-4.448500,-0.018450,0.010437
7582,2023-03-10,390.989990,393.160004,384.320007,385.910004,385.910004,189105300,SPY,-4.392001,-4.404001,-0.014429,0.010488
7583,2023-03-13,381.809998,390.390015,380.649994,385.359985,385.359985,157790000,SPY,-4.495003,-4.392001,-0.001425,0.010463
7584,2023-03-14,390.500000,393.450012,387.049988,391.730011,391.730011,149752400,SPY,-3.893004,-4.495003,0.016530,0.010763


In [4]:
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward', 'dones'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        '''
        Save a transition
        '''
        self.memory.append(Transition(*args))
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [6]:
class DQN(nn.Module):
    
    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)
    

In [10]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        # Initialize the DQN
        self.dqn = DQN(state_size, action_size)
        # Define the target network
        self.target_dqn = DQN(state_size, action_size)

        # Copy the weights from the DQN network to the target network
        self.update_target_network()

        self.memory = ReplayMemory(100000)
        
        # Define the device to use for training
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.dqn.to(self.device)
        
        # Define the exploration parameters
        self.epsilon = 1.0
        self.epsilon_decay = 0.9995
        self.epsilon_min = 0.01
        
    def select_action(self, state):
        with torch.no_grad():
            # Convert the state to a PyTorch tensor
            state = torch.FloatTensor(state).unsqueeze(0).to(self.device)

            # Compute the Q-values for the state
            q_values = self.dqn(state)
            

            # Split the Q-values into two halves
            half_size = int(self.action_size / 2)
            q1, q2 = q_values.split(half_size, dim=1)
            print(q1)
            print(q2)

            # Compute the index of the action with the highest Q-value for each half
            action1 = q1.argmax().item()
            action2 = q2.argmax().item()

            if action2 == 0: print(action1, action2)


        # Decay the exploration parameter
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

        # Choose a random action with probability epsilon
        if np.random.rand() < self.epsilon:
            action1 = np.random.randint(half_size)
            action2 = np.random.randint(half_size) + half_size
        
        return action1, action2
    
    def learn(self, optimizer, batch_size, gamma=0.99):
        # Sample a batch of experiences from the replay memory
        experiences = self.memory.sample(batch_size)

        batch = Transition(*zip(*experiences))
        states = torch.tensor(batch.state, dtype=torch.float32)
        actions = torch.tensor(batch.action, dtype=torch.int64)
        next_states = torch.tensor(batch.next_state, dtype=torch.float32)
        rewards = torch.tensor(batch.reward, dtype=torch.float32)
        dones = torch.tensor(batch.dones, dtype=torch.float32)
        

        # Compute the Q-values for the current state-action pairs using the DQN network
        current_q_values = self.dqn(states).gather(1, actions).squeeze(1)
        
        A = self.target_dqn(next_states)
        # Reshape A to have shape (128, 2, 9)
        A_reshaped = A.view(128, 2, 9)

        # Take the maximum value along the last dimension (which has size 9)
        B, _ = A_reshaped.max(dim=-1)
        
        # Compute the Q-values for the next states using the target network
        next_q_values = B
        rewards = rewards.unsqueeze(1)
        rewards = torch.cat((rewards, rewards), dim=1)
        # Compute the expected Q-values using the Bellman equation
        expected_q_values = rewards + gamma * next_q_values
        

        # Compute the loss between the current Q-values and the expected Q-values
        loss = F.smooth_l1_loss(current_q_values, expected_q_values)
        
        # Update the DQN network
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    def update_target_network(self):
        # Copy the weights from the DQN network to the target network
        self.target_dqn.load_state_dict(self.dqn.state_dict())

from environments.gym_env import TradingEnv
input_feature_list = [
    'open',
    'high',
    'low',
    'close',
    'adjclose',
    'signal',
    'signal+1',
    'volume',
    'volatility'
]
df = df.dropna()
# df = df[-1000:]
env = TradingEnv(df=df, input_feature_list=input_feature_list)

import torch.optim as optim




agent = DQNAgent(len(input_feature_list) + 6, 18)

# Set up the optimizer
optimizer = optim.Adam(agent.dqn.parameters(), lr=0.001)

# Run the environment for one episode
state = env.reset()
done = False


episode = 0
batch_size = 128
action_list = []
for episode in range(10):
    total_reward = 0
    while not done:
        # Select an action using the DQN agent
        action1, action2 = agent.select_action(state)
        
        action_list.append((action1, action2))

        # Take the action in the environment
        next_state, reward, done, info = env.step([action1, action2])

        # Add the experience to the replay memory
        agent.memory.push(state, [action1, action2], next_state, reward, done)

        # Move to the next state
        state = next_state
        total_reward += reward

        # Update the DQN agent
        if len(agent.memory) > batch_size:
            agent.learn(optimizer, batch_size)
    

    # Print the total reward for the episode
    print("Episode {}: Total reward = {}".format(episode, total_reward))
    print(max([f[0] for f in action_list]))
    print(min([f[0] for f in action_list]))
    print(max([f[1] for f in action_list]))
    print(min([f[1] for f in action_list]))

        # Update the target network every 10 episodes
    if episode % 2 == 0:
        agent.update_target_network()


    state = env.reset()
    print(state)




c:\Users\alexr\miniconda3\envs\TradeMaster\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
c:\Users\alexr\miniconda3\envs\TradeMaster\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float64
  logger.warn(
c:\Users\alexr\miniconda3\envs\TradeMaster\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\alexr\miniconda3\envs\TradeMaster\lib\site-packages\numpy\core\_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
c:\Users\alexr\miniconda3\envs\TradeMaster\lib\site-packages\numpy\core\_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


tensor([[ 24511.8008,  22624.5039,    510.1743, -16773.2637,  16753.7402,
         -70398.2578, -13074.1494,  29718.0664,    402.5260]])
tensor([[ 13074.2539,  64381.9141,  18462.1758,  32347.7031,   2354.2117,
          -7682.3413, -33217.9531, -10896.8174, -19859.8125]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[ 24588.6934,  67808.7344, -18020.7617, -11376.9414,  10019.6904,
         -68787.1562, -22943.3770,   4039.0027,  -6660.0884]])
tensor([[ 15859.8398,  77511.8047,  42789.8203,  16410.3438,  25585.7559,
         -37228.4648, -52192.2148, -10366.6963, -34938.8672]])
tensor([[ 25970.4727,  52191.4609, -14755.5635, -15868.9453,  13021.1328,
         -52274.9766, -13495.7520,  14107.8301,  -5083.5132]])
tensor([[ 15441.1826,  60453.8203,  26722.1680,   9999.0215,  13518.1719,
         -28388.9316, -39215.2773,  -5692.5537, -23759.3730]])
tensor([[ 22690.4727,  36483.3750,  -6173.9414, -14346.5049,  

c:\Users\alexr\miniconda3\envs\TradeMaster\lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
c:\Users\alexr\miniconda3\envs\TradeMaster\lib\site-packages\numpy\core\_methods.py:247: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, na

c:\Users\alexr\Quantitative_Finance_Modelling\environments\gym_env.py:157: RuntimeWarning: overflow encountered in double_scalars
  self.cash -= execution_price * order_quantity
c:\Users\alexr\Quantitative_Finance_Modelling\environments\gym_env.py:147: RuntimeWarning: invalid value encountered in double_scalars
  max_long_position = self.portfolio_value // order_price
c:\Users\alexr\Quantitative_Finance_Modelling\environments\gym_env.py:148: RuntimeWarning: invalid value encountered in double_scalars
  max_short_position = -self.portfolio_value // order_price
c:\Users\alexr\miniconda3\envs\TradeMaster\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
0 0
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan]])
tensor([[nan, na